# 3 Linear Regression - Curve Fitting (TensorFlow)
In this tutorial we will use two methods to fit a curve using [TensorFlow](https://www.tensorflow.org/):
- Direct solution using the least-squares method - this is the same method used in the previous tutorial with NumPy 
- Iterative optimisation using [stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent/)

## 3.1 Data
First, as before, we sample $n$ observed data from the underlying polynomial defined by weights $w$:

In [ ]:
import random
import numpy as np

# get ground-truth data from the "true" model 
n = 100 
w = [4, 3, 2, 1]
x = np.linspace(-1,1,n)[:,np.newaxis]
t = np.matmul(np.power(np.reshape(x,[-1,1]), 
                       np.linspace(len(w)-1,0,len(w))), w)
std_noise = 0.2
t_observed = np.reshape(
    [t[idx]+random.gauss(0,std_noise) for idx in range(n)],
    [-1,1])

## 3.2 Computation Graph and Session
[Graphs and sessions](https://www.tensorflow.org/guide/graphs) are important features of TensorFlow. Briefly:
1. a **graph** needs to be built to specify what computations are required; the graphs are made up of nodes (mathematical operations) and edges (multidimensional data arrays, i.e. *tensors*, that are consumed or produced by a computation)
1. **sessions** are then constructed to specify what computation to run -  e.g., what data to use and in what order. 

To facilitate the data feeding, [**placeholders**](https://www.tensorflow.org/api_docs/python/tf/placeholder) are used. The following two methods to fit the model provide two examples of how these are used in practice.

First, we build a computation graph using "tf functions":

In [ ]:
import tensorflow as tf


# placeholders are used for feeding data in runtime
ph_x = tf.placeholder(tf.float32, [n, 1])
ph_t = tf.placeholder(tf.float32, [n, 1])

deg = 3
# define the computation node X
node_X = tf.pow(ph_x, tf.linspace(tf.to_float(deg),0,deg+1))

This above is a very simple computation graph to evaluate the polynomial using TensorFlow functions. This can be built without any real data and there has not been any computation taking place either.  

Then we construct a session. And, call the run method to evaluate the node *node_X* to actually run the computation and obtain the results.

## 3.3 Least-Squares Solution

To begin this curve fitting tutorial, we will start with the same method from the previous NumPy tutorial: the least squares solution. The advantages of using TensorFlow are not obvious with this method, however they will become apparent later for SGD. 

In [ ]:
# we complete the computation graph with the least-square solution
node_w = tf.matrix_solve_ls(node_X, ph_t)

# run the session to evaluate the node weights
sess = tf.Session()  
dataFeed = {ph_x:x, ph_t:t_observed}  # feed data
w_lstsq = sess.run(node_w, feed_dict=dataFeed)
print(w_lstsq)
#tf.reset_default_graph()
sess.close()


## 3.3 Stochastic Gradient Descend Method
Instead of using least-squares, weights can be optimised by minimising a loss function between the predicted- and observed target values using [SGD](https://en.wikipedia.org/wiki/Stochastic_gradient_descent). SGD works by taking the gradient of a random point in our data set and using its value to inform whether to increase or decrease the value of the model weights.

It is not an efficient method for this curve fitting problem; this is only for the purpose of demonstrating how an iterative method can be implemented in TensorFlow.

In [ ]:
# build a new graph
ph_1x = tf.placeholder(tf.float32, [1, 1])
ph_1t = tf.placeholder(tf.float32, [1, 1])

deg = 3
node_X = tf.pow(ph_1x, tf.linspace(tf.to_float(deg),0,deg+1))

# first declare variables that need optimisation
var_w = tf.get_variable('weights', shape=[deg+1,1], 
                        initializer=tf.random_normal_initializer(0, 1e-3))

# complete the computation graph with SGD
node_1t = tf.matmul(node_X, var_w)
# define a square loss function
loss = tf.reduce_mean(tf.square(node_1t-ph_1t))
# buiding a train-op to minimise the loss
train_op = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# launch a session
sess = tf.Session()  
sess.run(tf.global_variables_initializer())  # initialise all the variables

# iteration to update variables with backprop gradients
total_iter = int(1e4)
indices_train = [i for i in range(n)]
for step in range(total_iter):

    idx = step % n
    if idx == 0:  # shuffle every epoch
        random.shuffle(indices_train)
    
    # single data point feed
    singleDataFeed = {
        ph_1x:x[indices_train[idx],np.newaxis], 
        ph_1t:t_observed[indices_train[idx],np.newaxis] }
    
    # print for testing
    #print(singleDataFeed)
    
    # update the variables
    sess.run(train_op, feed_dict=singleDataFeed)
    
    # print training information
    if (step % 200) == 0:
        loss_train = sess.run(loss, feed_dict=singleDataFeed)
        print('Step %d: Loss=%f' % (step, loss_train))
    if (step % 2000) == 0:
        w_sgd = sess.run(var_w)
        print('Estimated weights:')
        print(w_sgd)

w_sgd = sess.run(var_w)
print('Final weights at step %d:' % step)
print(w_sgd)
sess.close()

# Playing around with the optimiser



In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import numpy as np

%matplotlib inline

# get ground-truth data from the "true" model 
n = 100 
w = [4, 3, 2, 1]
x = np.linspace(-1,1,n)[:,np.newaxis]
t = np.matmul(np.power(np.reshape(x,[-1,1]), 
                       np.linspace(len(w)-1,0,len(w))), w)
std_noise = 0.2
t_observed = np.reshape(
    [t[idx]+random.gauss(0,std_noise) for idx in range(n)],
    [-1,1])

loss_val = np.array([])

# build a new graph
ph_1x = tf.placeholder(tf.float32, [1, 1])
ph_1t = tf.placeholder(tf.float32, [1, 1])

deg = 3
node_X = tf.pow(ph_1x, tf.linspace(tf.to_float(deg),0,deg+1))

# first declare variables that need optimisation
var_w = tf.get_variable('weights', shape=[deg+1,1], 
                        initializer=tf.random_normal_initializer(0, 1e-3))

# complete the computation graph with SGD
node_1t = tf.matmul(node_X, var_w)
# define a square loss function
loss = tf.reduce_mean(tf.square(node_1t-ph_1t))
# buiding a train-op to minimise the loss
train_op = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# launch a session
sess = tf.Session()  
sess.run(tf.global_variables_initializer())  # initialise all the variables

# iteration to update variables with backprop gradients
total_iter = int(1e1)
indices_train = [i for i in range(n)]
for step in range(total_iter):

    idx = step % n
    if idx == 0:  # shuffle every epoch
        random.shuffle(indices_train)
    
    # single data point feed
    singleDataFeed = {
        ph_1x:x[indices_train[idx],np.newaxis], 
        ph_1t:t_observed[indices_train[idx],np.newaxis] }
    
    
    # update the variables
    sess.run(train_op, feed_dict=singleDataFeed)
    
    if (step % 1) == 0:
        loss_train = sess.run(loss, feed_dict=singleDataFeed)
        np.append(loss_val,loss_train)
        print(loss_val)
        #plt.plot(range(step+1),loss_val)
        #plt.draw()
    
    # print training information
    if (step % 200) == 0:
        loss_train = sess.run(loss, feed_dict=singleDataFeed)
        print('Step %d: Loss=%f' % (step, loss_train))
    if (step % 2000) == 0:
        w_sgd = sess.run(var_w)
        print('Estimated weights:')
        print(w_sgd)

w_sgd = sess.run(var_w)
print('Final weights at step %d:' % step)
print(w_sgd)
sess.close()

[]
Step 0: Loss=1.570971
Estimated weights:
[[0.0018657 ]
 [0.00728675]
 [0.04763701]
 [0.31491482]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
Final weights at step 9:
[[1.1129044 ]
 [0.75512886]
 [1.8949149 ]
 [1.2097962 ]]


## Questions
- What happens to the convergence when you try other optimisation hyperparameters, such as a different optimiser, learning rate and number of iterations? You could try plotting the loss as a function of step for each hypermarater. 
- Try adding regularisers and different loss functions. How does this affect the performance?
- Would batch gradient descent or minibatch gradient descent improve the optimisation?
- Would higher-degree models be more prone to overfitting and why?
